BLDG 5301 Project

In [ ]:
import pandas as pd

# Load the provided AHU and weather data
ahu_file_path = "canal ahu 1.csv"
weather_file_path = "weather.csv"

# Read the AHU data
ahu_data = pd.read_csv(ahu_file_path)
# Read the weather data
weather_data = pd.read_csv(weather_file_path)

# Step 1: Rename and convert timestamp columns to datetime format
ahu_data.rename(columns={"Unnamed: 0": "Timestamp"}, inplace=True)
ahu_data["Timestamp"] = pd.to_datetime(ahu_data["Timestamp"])
weather_data.rename(columns={"Unnamed: 0": "Timestamp"}, inplace=True)
weather_data["Timestamp"] = pd.to_datetime(weather_data["Timestamp"])

# Step 2: Set the Timestamp column as the index for both datasets
ahu_data.set_index("Timestamp", inplace=True)
weather_data.set_index("Timestamp", inplace=True)

# Step 3: Reindex the AHU data to match the timestamps in the weather data
# This ensures that AHU data has the same timestamps as the weather data
ahu_data = ahu_data.reindex(weather_data.index)

# Step 4: Identify and print missing timestamps
missing_timestamps = ahu_data[ahu_data.isnull().any(axis=1)].index
if not missing_timestamps.empty:
    print("Missing Timestamps before cleaning:")
    print(missing_timestamps)

# Step 5: Convert all columns in AHU data to numeric where possible
# This step avoids errors during interpolation
ahu_data = ahu_data.apply(pd.to_numeric, errors="coerce")

# Step 6: Perform linear interpolation for missing values in AHU data
# This fills missing values in AHU data using linear interpolation
ahu_data_interpolated = ahu_data.interpolate(method="linear", axis=0)

# Step 7: Reset the index to prepare the cleaned AHU data for saving
# Clear the index name to avoid duplication of the Timestamp column
ahu_data_interpolated.index.name = None
ahu_data_cleaned = ahu_data_interpolated.reset_index()

# Step 8: Save the cleaned AHU data to a new CSV file
output_file_path = "canal_ahu1_clean.csv"
ahu_data_cleaned.to_csv(output_file_path, index=False)

# Output the path of the saved file for reference
print(f"Cleaned data saved to: {output_file_path}")


Missing Timestamps before cleaning:
DatetimeIndex(['2017-12-31 01:00:00', '2017-12-31 02:00:00',
               '2017-12-31 03:00:00', '2017-12-31 04:00:00',
               '2017-12-31 05:00:00', '2017-12-31 06:00:00',
               '2017-12-31 07:00:00', '2017-12-31 08:00:00',
               '2017-12-31 09:00:00', '2017-12-31 10:00:00',
               '2017-12-31 11:00:00', '2017-12-31 12:00:00',
               '2017-12-31 13:00:00', '2017-12-31 14:00:00',
               '2017-12-31 15:00:00', '2017-12-31 16:00:00',
               '2017-12-31 17:00:00', '2017-12-31 18:00:00',
               '2017-12-31 19:00:00', '2017-12-31 20:00:00',
               '2017-12-31 21:00:00', '2017-12-31 22:00:00',
               '2017-12-31 23:00:00'],
              dtype='datetime64[ns]', name='Timestamp', freq=None)
Cleaned data saved to: canal_ahu1_clean.csv
